# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
# from data_processing import DataProcessing

In [2]:
load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [4]:
load_nlp_model = spacy.load("en_core_web_md")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.948739442627313

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8743882813030235

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8743882813030235

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8511686894801594

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.9399013747850621

    Observation: Apple's stock rose by 900%.
    Similarity: 0.9199637743259186

    Observation: Apple's stock rose by 10%.
    Similarity: 0.9399013747850621



In [5]:
pom_mappings

{"Tesla's stock will rise by 10%.": {"Apple's stock will rise by 10%.": [0.948739442627313],
  "Apple's stock rose by 11%.": [0.8743882813030235],
  "Apple's stock rose by 10%.": [0.8743882813030235],
  "Apple's stock rose by 900%.": [0.8511686894801594]},
 "Apple's stock will rise by 10%.": {"Apple's stock will rise by 10%.": [1.0],
  "Apple's stock rose by 11%.": [0.9399013747850621],
  "Apple's stock rose by 900%.": [0.9199637743259186],
  "Apple's stock rose by 10%.": [0.9399013747850621]}}

In [6]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.948739
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.874388
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.874388
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.851169
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.939901
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.919964
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.939901


In [8]:
df[2]['Observation'].vector = df[4]['Observation'].vector

KeyError: 4

Comparison
1. No focus on #s, more on words

In [ ]:
prediction_doc = load_nlp_model("10%")
observation_doc = load_nlp_model("10.5%")
observation_doc_2 = load_nlp_model("11%")
similarity = prediction_doc.similarity(observation_doc)
similarity

In [ ]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

In [9]:
prediction_doc = load_nlp_model("Apple's stock rose by 10%.")
observation_doc = load_nlp_model("Apple's stock rose by 11%.")
similarity = prediction_doc.similarity(observation_doc)
similarity

0.9999999324834256

In [12]:
prediction_doc.vector

array([-8.01272571e-01,  1.79031253e-01, -5.82190007e-02, -4.93226722e-02,
       -1.86806634e-01,  1.06649976e-02, -2.27949008e-01,  2.23186314e-02,
        5.82532510e-02,  1.43173873e+00, -1.91523328e-01,  9.29074958e-02,
       -2.82150000e-01, -6.67073764e-03, -1.25323206e-01, -1.95667252e-01,
       -2.26755753e-01,  8.04331779e-01, -1.62955493e-01, -1.96525007e-02,
       -1.70191228e-02,  1.71808768e-02, -7.93302506e-02,  4.61874530e-04,
        1.48741268e-02,  2.48367470e-02, -1.25513121e-01,  1.32563382e-01,
       -1.96952522e-02, -1.96085006e-01, -1.53880164e-01,  1.51512586e-03,
        1.70807645e-01, -5.63037433e-02,  2.55896240e-01,  7.37582445e-02,
       -1.10983491e-01, -1.09516317e-02,  7.29376301e-02, -3.32989991e-02,
        4.64804769e-02, -2.20928505e-01,  2.63958365e-01,  2.69109011e-02,
        7.65738785e-02,  1.32258952e-01, -1.20638870e-01, -4.43828776e-02,
       -6.20492510e-02, -3.91898751e-02, -1.50005758e-01,  1.21044993e-01,
       -5.48769608e-02, -

In [13]:
observation_doc.vector

array([-8.01272571e-01,  1.79031253e-01, -5.82190007e-02, -4.93226722e-02,
       -1.86806634e-01,  1.06649976e-02, -2.27949008e-01,  2.23186314e-02,
        5.82532510e-02,  1.43173873e+00, -1.91523328e-01,  9.29074958e-02,
       -2.82150000e-01, -6.67073764e-03, -1.25323206e-01, -1.95667252e-01,
       -2.26755753e-01,  8.04331779e-01, -1.62955493e-01, -1.96525007e-02,
       -1.70191228e-02,  1.71808768e-02, -7.93302506e-02,  4.61874530e-04,
        1.48741268e-02,  2.48367470e-02, -1.25513121e-01,  1.32563382e-01,
       -1.96952522e-02, -1.96085006e-01, -1.53880164e-01,  1.51512586e-03,
        1.70807645e-01, -5.63037433e-02,  2.55896240e-01,  7.37582445e-02,
       -1.10983491e-01, -1.09516317e-02,  7.29376301e-02, -3.32989991e-02,
        4.64804769e-02, -2.20928505e-01,  2.63958365e-01,  2.69109011e-02,
        7.65738785e-02,  1.32258952e-01, -1.20638870e-01, -4.43828776e-02,
       -6.20492510e-02, -3.91898751e-02, -1.50005758e-01,  1.21044993e-01,
       -5.48769608e-02, -